In [ ]:
# Importing necessary libraries

import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import seaborn as sns

In [ ]:
# Loading in gene/TF expression full data

gene_expression = pd.read_csv(('/Users/christianlangridge/Desktop/Zhang-Lab/Zhang Lab Data/Full data files/Geneexpression (full).tsv'), sep='\t', header=0)
tf_expression = pd.read_csv(('/Users/christianlangridge/Desktop/Zhang-Lab/Zhang Lab Data/Full data files/TF(full).tsv'), sep='\t', header=0)

In [ ]:
# Split into training, testing and validation sets and into numpy arrays + combining dataframes
x = tf_expression
y = gene_expression

combined_data = pd.concat([x, y], axis=1)

# First split: 70% train and 30% temp (test + val)
x_train, x_temp, y_train, y_temp = train_test_split(
    x, y, test_size=0.3, random_state=42)

# Second split: split the temp set into 20% test and 10% val (which is 2/3 and 1/3 of temp)
x_test, x_val, y_test, y_val = train_test_split(
    x_temp, y_temp, test_size=1/3, random_state=42)

# For training set
x_train = x_train.to_numpy()
y_train = y_train.to_numpy()

# For validation set
x_val = x_val.to_numpy()
y_val = y_val.to_numpy()

x_test = x_test.to_numpy()
y_test = y_test.to_numpy()

In [ ]:
# out-of-the-box r2 score with ElasticNet: 0.00432651511965587
# no tuning at all so not surprised by uncompetitive results vs. MLR

from sklearn.linear_model import ElasticNet
elas_reg = ElasticNet().fit(x_train,y_train)
print('Score ', elas_reg.score(x_test,y_test))    

In [ ]:
# tuning performance of ElasticNet
# scaling input features 

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import MultiTaskElasticNetCV

scaler = StandardScaler()
X_train_std = scaler.fit_transform(x_train)
X_test_std  = scaler.transform(x_test)

In [ ]:
# L1/L2 penalty tuning with ElasticNetCV

elas = MultiTaskElasticNetCV(
    alphas=np.logspace(-4, 1, 50),
    l1_ratio=[0.1, 0.3, 0.5, 0.7, 0.9],
    cv=5,
    n_jobs=-1,
    max_iter=10000,
)
elas.fit(X_train_std, y_train)
print("Test R2:", elas.score(X_test_std, y_test))
print("alpha_:", elas.alpha_, "l1_ratio_:", elas.l1_ratio_)